## Kelvin-Voigt - parallel

In [ ]:
from bmcs_utils.api import Cymbol
import sympy as sp
sp.init_printing()

In [ ]:
sig = Cymbol(r'\sigma', codename='sig')
eps = Cymbol(r'\varepsilon', codename='eps')
dot_eps = Cymbol(r'\dot{\varepsilon}', codename='dot_eps')

In [ ]:
t = Cymbol(r't')
tau = Cymbol(r'\tau', codename='tau')
eps_t = sp.Function(r'epsilon')(t)
dot_eps_t = eps_t.diff(t)

In [ ]:
eps_t, dot_eps_t

In [ ]:
E = Cymbol(r'E')
eta = Cymbol(r'\eta', codename='eta')

In [ ]:
sig_t = E * eps_t + eta * dot_eps_t
sig_t

In [ ]:
Phi_t = sp.integrate(sig_t, (eps_t, 0, eps))
Phi_t


## Maxwell - serial

In [ ]:
eps_e = Cymbol(r'\varepsilon_\mathrm{e}', codename='eps_e')
eps_v = Cymbol(r'\varepsilon_\mathrm{v}', codename='eps_e')


In [ ]:
# sig = Cymbol(r'\sigma', codename='sig')
# dot_sig = Cymbol(r'\dot{\sigma}', codename='dot_sig')
sig_t = sp.Function(r'sigma')(t)
dot_sig_t = sig_t.diff(t)

In [ ]:
eps_t_ = sig_t / E + dot_sig_t / eta
eps_t_

In [ ]:
G_t = sp.integrate(eps_t_, sig_t)
G_t


In [ ]:
G_t.diff(sig_t)

In [ ]:
sp.simplify(sp.solve(sp.Eq(eps_t, G_t.diff(sig_t)), sig_t)[0])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Material properties
E = 210e9  # Young's modulus in Pascals (Pa)
eta = 20*E  # Viscosity in Pa.s

# Simulation parameters
time_steps = 1000
total_time_slow = 100  # Slow process over 100 seconds
total_time_fast = 10   # Fast process over 10 seconds
target_strain = 0.001  # Total strain

# Time increments for each process
dt_slow = total_time_slow / time_steps
dt_fast = total_time_fast / time_steps

# Create time arrays
times_slow = np.linspace(0, total_time_slow, time_steps)
times_fast = np.linspace(0, total_time_fast, time_steps)

# Define strain rates (constant rate loading) for slow and fast processes
slow_rate = target_strain / total_time_slow
fast_rate = target_strain / total_time_fast

# Compute strain at each time step
strain_slow = slow_rate * times_slow
strain_fast = fast_rate * times_fast

# Initializations
stress_slow = np.zeros(time_steps)
stress_fast = np.zeros(time_steps)

strain_e_slow = np.zeros(time_steps)
strain_v_slow = np.zeros(time_steps)

strain_e_fast = np.zeros(time_steps)
strain_v_fast = np.zeros(time_steps)

# Time-stepping scheme for slow and fast rates
for t in range(1, time_steps):
    # Slow loading
    delta_strain_slow = strain_slow[t] - strain_slow[t-1]
    strain_e_slow[t] = (strain_e_slow[t-1] + delta_strain_slow) / (1 + dt_slow * E / eta)
    strain_v_slow[t] = strain_slow[t] - strain_e_slow[t]
    stress_slow[t] = E * strain_e_slow[t]

    # Fast loading
    delta_strain_fast = strain_fast[t] - strain_fast[t-1]
    strain_e_fast[t] = (strain_e_fast[t-1] + delta_strain_fast) / (1 + dt_fast * E / eta)
    strain_v_fast[t] = strain_fast[t] - strain_e_fast[t]
    stress_fast[t] = E * strain_e_fast[t]

# Plotting the results
plt.figure()
plt.plot(strain_slow, stress_slow, label='Slow Rate')
plt.plot(strain_fast, stress_fast, label='Fast Rate')
plt.xlabel('Strain')
plt.ylabel('Stress (Pa)')
plt.title('Maxwell Model - Strain-Controlled Process with Different Rates')
plt.legend()
plt.grid(True)
plt.show()

## Exemplary algorithm for viscoplasticity

In [ ]:
import sympy as sp
from bmcs_utils.api import Cymbol
import numpy as np
from scipy.optimize import minimize_scalar
from matplotlib.pylab import plt

In [ ]:
# Define symbols
E, eta = sp.symbols('E eta', real=True, positive=True)  # Elastic modulus and viscosity
epsilon, epsilon_e, epsilon_v = sp.symbols('epsilon epsilon_e epsilon_v', real=True)
dot_epsilon_v = Cymbol(r'\dot{\varepsilon}_\mathrm{v}', codename='dot_epsilon_v', real=True)
sigma = sp.symbols('sigma', real=True)

epsilon_e_ = epsilon - epsilon_v
# Define potentials
Psi_ = sp.Rational(1, 2) * E * epsilon_e_**2
Phi_ = sp.Rational(1, 2) * eta * dot_epsilon_v**2
# Derive stress from free energy potential
sigma_ = sp.diff(Psi_, epsilon) 
sigma_v_ = sp.diff(Psi_, epsilon_v)
sp.simplify(sigma_), sp.simplify(sigma_v_)
# Derive evolution of viscous strain rate from dissipation potential
dot_epsilon_v_ = sigma / eta

# Lamdify expressions for numerical computation
get_sigma = sp.lambdify((epsilon, epsilon_v, E, eta), sigma_, modules='numpy', cse=True)
get_Psi = sp.lambdify((epsilon, epsilon_v, E, eta), Psi_, modules='numpy')
get_Phi = sp.lambdify((dot_epsilon_v, E, eta), Phi_, modules='numpy')
# get_epsilon_v = sp.lambdify(sigma, dot_epsilon_v_, modules='numpy')

# Material parameters
E_val = 100.0  # Example Elastic modulus
eta_val = 10.0  # Example Viscosity

# Time-stepping setup
time_steps = 11
total_time = 1 # 10.0
delta_t = total_time / time_steps
strain_history = np.linspace(0, 0.1, time_steps)  # Monotonic increasing strain
sigma_values = np.zeros(time_steps)
epsilon_v_values = np.zeros(time_steps)

# Initial conditions
sigma_values[0] = 0
epsilon_v_values[0] = 0

# Time integration loop
for i in range(1, time_steps):
    _epsilon = strain_history[i]

    # Define minimization problem
    def incremental_potential(epsilon_v_next):
        
        # Numerically evaluate psi and sigma
        dot_epsilon_v = (epsilon_v_next - epsilon_v_values[i-1]) / delta_t
        
        delta_Psi = (get_Psi(_epsilon, epsilon_v_next, E_val, eta_val) - 
                    get_Psi(_epsilon, epsilon_v_values[i-1], E_val, eta_val))
        delta_Phi = delta_t * get_Phi(dot_epsilon_v, E_val, eta_val)
        
        return delta_Psi + delta_Phi

    # Solve minimization problem
    result = minimize_scalar(
        incremental_potential,
        bounds=(epsilon_v_values[i-1], _epsilon),
        method='bounded'
    )

    # Update values
    epsilon_v_values[i] = result.x
    sigma_values[i] = get_sigma(_epsilon, result.x, E_val, eta_val)


# Output results
for i in range(time_steps):
    print(f"Step {i}: epsilon={strain_history[i]}, sigma={sigma_values[i]}, epsilon_v={epsilon_v_values[i]}")

In [ ]:
plt.plot(strain_history, sigma_values);

## Stress driven dissipation

In [ ]:
# Define symbols
E, eta = sp.symbols('E eta', real=True, positive=True)  # Elastic modulus and viscosity
epsilon, epsilon_e, epsilon_v = sp.symbols('epsilon epsilon_e epsilon_v', real=True)
#dot_epsilon_v = Cymbol(r'\dot{\varepsilon}_\mathrm{v}', codename='dot_epsilon_v', real=True)
sigma = sp.symbols('sigma', real=True)

In [ ]:

epsilon_e_ = epsilon - epsilon_v
# Define potentials
Psi_ = sp.Rational(1, 2) * E * epsilon_e_**2
Phi_ = sp.Rational(1, 2) * sigma**2 / eta
# Derive stress from free energy potential
sigma_ = sp.diff(Psi_, epsilon)
sigma_, Phi_.diff(sigma)

In [ ]:
# Derive evolution of viscous strain rate from dissipation potential
dot_epsilon_v_ = Phi_.diff(sigma)
Psi_ + t * Phi_.subs(sigma, sigma_)

$$
\Delta \Pi \approx \frac{d\Psi}{dt} + \frac{d}{dt}\left(\int \Phi \, dt\right)
$$

$$
\frac{d\Psi}{dt} + \Phi(\dot{\varepsilon}_v) \geq 0
$$

In [ ]:
t = sp.symbols('t', real=True)
eps_v_t_ = sp.Function('epsilon_V')(t)
delta_Pi = sp.simplify(Psi_.subs(epsilon_v, eps_v_t_) + t * Phi_.subs(sigma, sigma_).subs(epsilon_v, eps_v_t_))
dPi_ = delta_Pi.subs(eps_v_t_, dot_epsilon_v).diff(dot_epsilon_v)
dPi_, sp.solve(dPi_, dot_epsilon_v)

In [ ]:
Pi_ = Psi_ + Phi_ 
Pi_

In [ ]:
# Define symbols
E, eta = sp.symbols('E eta', real=True, positive=True)  # Elastic modulus and viscosity
epsilon, epsilon_e, epsilon_v = sp.symbols('epsilon epsilon_e epsilon_v', real=True)
dot_epsilon_v = Cymbol(r'\dot{\varepsilon}_\mathrm{v}', codename='dot_epsilon_v', real=True)
sigma = sp.symbols('sigma', real=True)

epsilon_e_ = epsilon - epsilon_v
# Define potentials
Psi_ = sp.Rational(1, 2) * E * epsilon_e_**2
Phi_ = sp.Rational(1, 2) * eta * dot_epsilon_v**2
# Derive stress from free energy potential
sigma_ = sp.diff(Psi_, epsilon) 
sigma_v_ = sp.diff(Psi_, epsilon_v)
sp.simplify(sigma_), sp.simplify(sigma_v_)
# Derive evolution of viscous strain rate from dissipation potential
dot_epsilon_v_ = sigma / eta

# Lamdify expressions for numerical computation
get_sigma = sp.lambdify((epsilon, epsilon_v, E, eta), sigma_, modules='numpy')
get_Psi = sp.lambdify((epsilon, epsilon_v, E, eta), Psi_, modules='numpy')
get_Phi = sp.lambdify((dot_epsilon_v, E, eta), Phi_, modules='numpy')
# get_epsilon_v = sp.lambdify(sigma, dot_epsilon_v_, modules='numpy')

##############

xPhi_ = sp.Rational(1, 2) * sigma**2 / eta
xget_Phi = sp.lambdify((sigma, E, eta), xPhi_, modules='numpy')
xdot_epsilon_v_ = xPhi_.diff(sigma)
xget_dot_epsilon_v = sp.lambdify((sigma, eta), xdot_epsilon_v_, modules='numpy')

##############
# Material parameters
E_val = 100.0  # Example Elastic modulus
eta_val = 10.0  # Example Viscosity

# Time-stepping setup
time_steps = 2 # 11
total_time = 1 # 10.0
delta_t = total_time / time_steps
strain_history = np.linspace(0, 0.1, time_steps)  # Monotonic increasing strain
sigma_values = np.zeros(time_steps)
epsilon_v_values = np.zeros(time_steps)

# Initial conditions
sigma_values[0] = 0
epsilon_v_values[0] = 0

# Time integration loop
for i in range(1, time_steps):
    _epsilon = strain_history[i]

    # Define minimization problem
    def incremental_potential(epsilon_v_next):
        # free energy
        delta_Psi = (get_Psi(_epsilon, epsilon_v_next, E_val, eta_val) - 
                    get_Psi(_epsilon, epsilon_v_values[i-1], E_val, eta_val))
        # dissipation increment - dot_epsilon_v
        dot_epsilon_v = (epsilon_v_next - epsilon_v_values[i-1]) / delta_t
        delta_Phi = delta_t * get_Phi(dot_epsilon_v, E_val, eta_val)
        # dissipation increment - sigma
        # sigma = get_sigma(_epsilon, epsilon_v_next, E_val, eta_val)
        # xdelta_Phi = delta_t * xget_Phi(sigma, E_val, eta_val)
        # print(eta_val * dot_epsilon_v, delta_Phi, sigma, xdelta_Phi)
        return delta_Psi + delta_Phi

    # Solve minimization problem
    result = minimize_scalar(
        incremental_potential,
        bounds=(epsilon_v_values[i-1], _epsilon),
        method='bounded'
    )

    # Update values
    epsilon_v_values[i] = result.x
    sigma_values[i] = get_sigma(_epsilon, result.x, E_val, eta_val)


# Output results
for i in range(time_steps):
    print(f"Step {i}: epsilon={strain_history[i]}, sigma={sigma_values[i]}, epsilon_v={epsilon_v_values[i]}")

$$

$$

$$
 \frac{\partial \mathcal{N}(\varepsilon_i)}{ \partial \varepsilon} - \sigma_i
$$

$$
\frac{1}{2}(
 \frac{\partial \mathcal{N}(\varepsilon_i)}{ \partial \varepsilon} 
 +
 \frac{\partial \mathcal{N}(\varepsilon_{i+1})}{ \partial \varepsilon} 
)
 -
 \frac{1}{2}(
 (
 \sigma_i + \sigma_{i+1}
 )
$$

$$
 \frac{\partial \mathcal{N}(\varepsilon_i)}{ \partial \varepsilon} 
 +
 \Delta \varepsilon_i
 \frac{\partial^2 \mathcal{N}(\varepsilon_{i})}{ \partial \varepsilon \partial \varepsilon} 
 -
  \sigma_{i+1}
$$

## Combined viscoelasticity and viscoplasticity

In [ ]:
import numpy as np
import sympy as sp
from scipy.optimize import minimize

# Define symbols for symbolic computation
E, eta_v, eta_p, sigma_y = sp.symbols('E eta_v eta_p sigma_y', real=True, positive=True)
epsilon, epsilon_v, epsilon_p, sigma = sp.symbols('epsilon epsilon_v epsilon_p sigma', real=True)

# Define symbolic expressions
epsilon_e_ = epsilon - epsilon_v - epsilon_p
Psi_ = sp.Rational(1, 2) * E * epsilon_e_**2
sigma_trial_ = sp.diff(Psi_, epsilon)

# Yield function for Perzyna model
f_ = sp.Abs(sigma) - sigma_y

# Dissipation potentials for symbolic computation
Phi_viscous_ = (1 / 2) * eta_v * epsilon_v**2  # Symbolic, epsilon_v is not rate here; adjust contextually
Phi_plastic_ = sp.Max(0, f_) / eta_p  # Using Perzyna viscoplastic flow

# Lambdify the expressions for numerical evaluation
get_Psi = sp.lambdify((epsilon_v, epsilon_p, epsilon, E), Psi_, modules='numpy')
get_sigma_trial = sp.lambdify((epsilon_v, epsilon_p, epsilon, E), sigma_trial_, modules='numpy')
get_f = sp.lambdify((sigma, sigma_y), f_, modules='numpy')
get_Phi_viscous = sp.lambdify((epsilon_v, eta_v), Phi_viscous_, modules='numpy')
get_Phi_plastic = sp.lambdify((sigma, eta_p, sigma_y), Phi_plastic_, modules='numpy')

# Example material parameters (numerical variables)
E_var = 100.0
eta_v_var = 10.0
eta_p_var = 5.0
sigma_y_var = 50.0

# Example loading (numerical variable)
epsilon_var = 0.05  # Total strain applied
epsilon_v_guess = 0.01
epsilon_p_guess = 0.01

# Objective function for minimizing energy potential
def energy_potential(epsilon_v_var, epsilon_p_var):
    sigma_trial_var = get_sigma_trial(epsilon_v_var, epsilon_p_var, epsilon_var, E_var)
    f_val = get_f(sigma_trial_var, sigma_y_var)
    phi_viscous_val = get_Phi_viscous(epsilon_v_var, eta_v_var)
    phi_plastic_val = get_Phi_plastic(sigma_trial_var, eta_p_var, sigma_y_var)
    delta_psi_val = get_Psi(epsilon_v_var, epsilon_p_var, epsilon_var, E_var)
    total_potential = delta_psi_val + phi_viscous_val + phi_plastic_val*f_val
    return total_potential

# Minimize the potential to find viscoelastic and viscoplastic strains
result = minimize(
    lambda x: energy_potential(x[0], x[1]),
    [epsilon_v_guess, epsilon_p_guess],
    bounds=[(0, epsilon_var), (0, epsilon_var)],
    method='L-BFGS-B'
)

epsilon_v_opt, epsilon_p_opt = result.x

print(f"Optimized epsilon_v: {epsilon_v_opt}, epsilon_p: {epsilon_p_opt}")

In [ ]:
import numpy as np
import sympy as sp
from scipy.optimize import minimize

# Define symbols for symbolic computation
E, eta_v, eta_p, sigma_y, dt = sp.symbols('E eta_v eta_p sigma_y dt', real=True, positive=True)
epsilon, epsilon_v, epsilon_p, sigma = sp.symbols('epsilon epsilon_v epsilon_p sigma', real=True)

# Define symbolic expressions
epsilon_e_ = epsilon - epsilon_v - epsilon_p
Psi_ = sp.Rational(1, 2) * E * epsilon_e_**2
sigma_trial_ = E * epsilon_e_

# Yield function for Perzyna model
f_ = sp.Abs(sigma) - sigma_y

# Define rate terms and dissipation potentials
dot_epsilon_v = epsilon_v / dt  # Assumes epsilon_v accumulation over a time increment dt
dot_epsilon_p = epsilon_p / dt  # Similar assumption for plastic strain

Phi_viscous_ = sp.Rational(1, 2) * eta_v * dot_epsilon_v**2
Phi_plastic_ = sp.Max(0, f_) / eta_p

# Lambdify the expressions for numerical evaluation
get_Psi = sp.lambdify((epsilon_v, epsilon_p, epsilon, E), Psi_, modules='numpy')
get_sigma_trial = sp.lambdify((epsilon_v, epsilon_p, epsilon, E), sigma_trial_, modules='numpy')
get_f = sp.lambdify((sigma, sigma_y), f_, modules='numpy')
get_Phi_viscous = sp.lambdify((epsilon_v, dt, eta_v), Phi_viscous_, modules='numpy')
get_Phi_plastic = sp.lambdify((sigma, eta_p, sigma_y), Phi_plastic_, modules='numpy')

In [ ]:
Phi_viscous_, Phi_plastic_

In [ ]:
# Example material parameters (numerical variables)
E_var = 100.0
eta_v_var = 10.0
eta_p_var = 10
sigma_y_var = 1.0

# Simulation parameters
time_total = 1.0
n_steps = 50
delta_time = time_total / n_steps
epsilon_arr = np.linspace(0, 0.05, n_steps)

epsilon_v_hist = np.zeros(n_steps)
epsilon_p_hist = np.zeros(n_steps)

# Objective function for minimizing energy potential
def energy_potential(epsilon_v_var, epsilon_p_var, epsilon_tot_var, delta_time_var):
    sigma_trial_var = get_sigma_trial(epsilon_v_var, epsilon_p_var, epsilon_tot_var, E_var)
    f_val = get_f(sigma_trial_var, sigma_y_var)
    phi_viscous_val = get_Phi_viscous(epsilon_v_var, delta_time_var, eta_v_var)
    phi_plastic_val = get_Phi_plastic(sigma_trial_var, eta_p_var, sigma_y_var)
    delta_psi_val = get_Psi(epsilon_v_var, epsilon_p_var, epsilon_tot_var, E_var)
    total_potential = delta_psi_val + delta_time_var * (phi_viscous_val + phi_plastic_val * f_val)
    return total_potential

# Time-stepping procedure
for step in range(1, n_steps):
    epsilon_tot_var = epsilon_arr[step]
    epsilon_v_guess = epsilon_v_hist[step - 1]
    epsilon_p_guess = epsilon_p_hist[step - 1]

    # Minimize the potential to find viscoelastic and viscoplastic strains
    result = minimize(
        lambda x: energy_potential(x[0], x[1], epsilon_tot_var, delta_time),
        [epsilon_v_guess, epsilon_p_guess],
        bounds=[(0, epsilon_tot_var), (0, epsilon_tot_var)],
        method='L-BFGS-B'
    )

    epsilon_v_hist[step], epsilon_p_hist[step] = result.x
    print(f"Step {step}: epsilon_v = {epsilon_v_hist[step]}, epsilon_p = {epsilon_p_hist[step]}")

# Output results for each time step
for step in range(n_steps):
    print(f"Time step {step}: strain = {epsilon_arr[step]}, epsilon_v = {epsilon_v_hist[step]}, epsilon_p = {epsilon_p_hist[step]}")

In [ ]:
sigma_arr = get_sigma_trial(epsilon_v_hist, epsilon_p_hist, epsilon_arr, E_var)
plt.plot(epsilon_arr, sigma_arr)
